# MDSTC-DBSCAN Demo

Data: https://data.ct.gov/Housing-and-Development/Real-Estate-Sales-2001-2020-GL/5mzw-sjtu

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from MDST_DBSCAN.MDSTC_DBSCAN.mdstc_dbscan import mdstcdbscan
from shapely.geometry import Polygon,MultiPoint, Point
from datetime import datetime
import numpy as np

In [ ]:
df = pd.read_csv(r'.\Data\realestatesales.csv')
df


In [ ]:
df = df[df['Location'].notna()]
df

In [ ]:
df['Date Recorded'] = pd.to_datetime(df['Date Recorded'])
filtered_df = df[(df['Date Recorded'].dt.year >= 2015) & (df['Date Recorded'].dt.year <= 2020)]

In [ ]:
property_types = filtered_df['Property Type'].value_counts()
filtered_df = filtered_df[filtered_df['Property Type'] == 'Single Family']


In [ ]:
from shapely.wkt import loads

# Convert the WKT coordinates to Shapely Point objects
filtered_df['geometry'] = filtered_df['Location'].apply(loads)

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(filtered_df, geometry='geometry')

center_point = loads('POINT(-72.71 41.5)')

# Plot the GeoDataFrame
fig, ax = plt.subplots()
gdf.plot(ax=ax, marker='o', color='red', markersize=5)

# Set axis labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Entries with Location Points')


ax.set_xlim(center_point.x - 1, center_point.x + 1)
ax.set_ylim(center_point.y - 1, center_point.y + 1)

# Display the plot
plt.show()

In [ ]:
# Convert the 'geometry' column to a GeoSeries
gdf = gpd.GeoSeries(filtered_df['geometry'])

# Extract x and y coordinates
x = gdf.geometry.x
y = gdf.geometry.y
time = filtered_df['Date Recorded']
value = filtered_df['Sale Amount']
ids = filtered_df.index.values
ids = pd.Series(filtered_df.index.values)


In [ ]:
eps = 0.2 #0.04
eps2 = 70000 #500
minpts = 300 #20
#eps=0.038, eps2=1000.0, minpts=110

model = mdstcdbscan(eps, eps2, minpts,1)
cluster_mark, df, reward, data = model.run(x, y, time, value, ids)



In [ ]:
colors = ['r', 'b', 'g', 'c', 'm', 'y', 'k', 'pink', 'orange', 'purple', 'brown','w']

# Create figure and axes
labels, counts = np.unique(df['cluster_mark'], return_counts=True)
n_clusters = len(labels)

for i, label in enumerate(labels):
    print(f"Cluster {label}: {counts[i]}")
fig, ax = plt.subplots(figsize=(10, 8))

# Iterate over each cluster
for i in range(n_clusters):
    # Get indices of points in the current cluster
    idx = np.where(df['cluster_mark'] == labels[i])[0]

    # Get x and y values for the points in the current cluster
    y_plot = df['y'][idx]
    x_plot = df['x'][idx]
    # Plot the points with the appropriate color
    ax.scatter(x_plot, y_plot, c=colors[i % len(colors)], alpha=0.5, label=f'Cluster {i+1}')


# Set labels for the axes
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

center_point = loads('POINT(-72.71 41.5)')


ax.set_xlim(center_point.x - 1, center_point.x + 1)
ax.set_ylim(center_point.y - 1, center_point.y + 1)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels)
plt.show()

##### Taking the major cluster mark of each location

In [ ]:
count_df = df.groupby(['x', 'y', 'cluster_mark']).size().reset_index(name='count')
count_df
max_cluster_df = count_df.loc[count_df.groupby(['x', 'y'])['count'].idxmax()][['x', 'y', 'cluster_mark']]



In [ ]:
count_df

In [ ]:
merged_df = df.merge(max_cluster_df, on=['x', 'y'], suffixes=('', '_max'))
merged_df

In [ ]:
# Create figure and axes
labels, counts = np.unique(merged_df['cluster_mark_max'], return_counts=True)
n_clusters = len(labels)

for i, label in enumerate(labels):
    print(f"Cluster {label}: {counts[i]}")
fig, ax = plt.subplots(figsize=(10, 8))


# Iterate over each cluster
for i in range(n_clusters):
    # Get indices of points in the current cluster
    idx = np.where(merged_df['cluster_mark_max'] == labels[i])[0]

    # Get x and y values for the points in the current cluster
    y_plot = merged_df['y'][idx]
    x_plot = merged_df['x'][idx]
    # Plot the points with the appropriate color
    ax.scatter(x_plot, y_plot, c=colors[i % len(colors)], alpha=0.5, label=f'Cluster {i+1}')

# Set labels for the axes
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

center_point = loads('POINT(-72.71 41.5)')


ax.set_xlim(center_point.x - 1, center_point.x + 1)
ax.set_ylim(center_point.y - 1, center_point.y + 1)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels)
plt.show()

#### Creating Convex Hulls

In [ ]:
# Create a list of Point objects from x and y columns
points = [Point(y,x) for y,x in zip(merged_df['y'], merged_df['x'])]

# Create a GeoDataFrame from merged_df and points
gdf = gpd.GeoDataFrame(merged_df, geometry=points)

In [ ]:

# Create a dictionary to hold the polygons for each cluster
polygons = {}

# Loop over the clusters and create a polygon for each one
for cluster in gdf.cluster_mark.unique():
    if cluster != -1:
        points = gdf[gdf.cluster_mark == cluster].geometry.to_list()
        if len(points) > 0:
            # Swap xmerged_df2['cluster1_max'] and y coordinates
            points = [(point.y, point.x) for point in points]
            # Create a convex hull around the points
            hull = MultiPoint(points).convex_hull
            polygons[cluster] = hull

# Create a new GeoDataFrame with the polygons
poly_gdf = gpd.GeoDataFrame(geometry=list(polygons.values()))



# Add the cluster labels to the GeoDataFrame
poly_gdf['cluster_mark_max'] = list(polygons.keys())
poly_gdf


In [ ]:
colors = ['red', 'blue', 'green', 'purple', 'orange', 'pink', 'brown', 'grey', 'teal', 'magenta', 'olive', 'navy', 'maroon','yellow','c','m']
#colors = ['r', 'b', 'g', 'c', 'm', 'y', 'k', 'pink', 'orange', 'purple', 'brown','w']

# Set the plot style and plot the polygons
#style_kwds = {'border': 'gray', 'linewidth': 1.5, 'edgecolor': 'black'}
fig, ax = plt.subplots(figsize=(12, 8))
shapefile.plot(ax=ax, facecolor='none', edgecolor='k')

for i, cluster in enumerate(polygons.keys()):
    poly_gdf[poly_gdf.cluster_mark_max == cluster].plot(ax=ax,color=colors[i % len(colors)], alpha=0.5,label=f'Cluster {i+1}')

    
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='lower right')

center_point = loads('POINT(-72.71 41.5)')


ax.set_xlim(center_point.x - 1, center_point.x + 1)
ax.set_ylim(center_point.y - 1, center_point.y + 1)


ax.set_title('Polygons by Cluster')
ax.set_aspect('equal')
plt.show()